<a href="https://colab.research.google.com/github/yangjik/yang.study/blob/main/11_%ED%8A%B8%EB%A6%AC_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

들어가기 앞서서 정형데이터 / 비정형데이터 확인하기

- 정형데이터 : 데이터가 구조를 가지고있다.

- 비전형데이터 : 데이터베이스나 엑셀로 표현하기 어려운데이터.


# 앙상블

앙상블은 결정트리 기반으로 만들어졌다.<br>
정형 데이터를 다루는데 가장 뛰어난다.

### 랜덤 포레스트
- 앙상블 학습의 대표
- 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻는다.

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
    )

In [10]:
# 교차 검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier   # 랜덤포레스트 분류모델

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1
                        )
print('train_score : ', np.mean(scores['train_score']), '\ntest_score : ', np.mean(scores['test_score']))

train_score :  0.9973541965122431 
test_score :  0.8905151032797809


In [11]:
# 위 결과 확인하면 과대적합 발생
rf.fit(train_input, train_target)
print('특성 중요도 : ', rf.feature_importances_)

특성 중요도 :  [0.23167441 0.50039841 0.26792718]


In [12]:
#  OOB샘플 - 부트스르탭 샘플로 훈련한 결정트리를 평가 -> 검증세트 역활
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
rf.oob_score_

0.8934000384837406

위에 교차검증한 점수랑 oob 점수랑 비슷하게 나왔다.

### 엑스트라 트리
- 기본적으로 100개의 결정트리 훈련
- DecisionTreeClassifier 의 매개변수 splitter = 'random'
- 하나의 결정 트리에서 특성을 무작위로 분할하면 성능은 낮아지지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증세트의 점수를 높힌다.

In [13]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
score = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.9974503966084433 0.8887848893166506


결정트리는 최적의 분할을 찾는 데 시간을 많이 소모한다. 특히 고려해야 할 특성의 개수가 많을 때 그렇다. 만약 무작위로 나눈다면 훨씬 빨리 트리를 구성한다.

In [14]:
 et.fit(train_input, train_target)
 et.feature_importances_
 # 밑에 결과 [알콜 도수, 당도, pH]

array([0.20183568, 0.52242907, 0.27573525])

### 그레이디언트 부스팅
- 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완
- 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반저긍로 높은 일반화 성능을 기대한다.

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
score = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [17]:
score

{'fit_time': array([0.52229929, 0.52154708, 0.56368184, 0.54603839, 0.36534309]),
 'score_time': array([0.00450253, 0.00424814, 0.00530601, 0.00483823, 0.00332713]),
 'test_score': array([0.86634615, 0.87019231, 0.89412897, 0.86044273, 0.86910491]),
 'train_score': array([0.89006495, 0.88958383, 0.88239538, 0.89249639, 0.88600289])}

In [18]:
print('train 스코어 : ', np.mean(score['train_score']), '\ntest 스코어 : ', np.mean(score['test_score']))

train 스코어 :  0.8881086892152563 
test 스코어 :  0.8720430147331015


위 결과를 보면 과대적합이 안되어있다.<br>
그레이디언트 부스팅은 결정 트리리 개수를 늘려도 과대적합에 강하다.<br>
그래서 학습률을 증가시키고 트리개수 늘리면 성능이 더 좋게 나온다.<br>

In [22]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
score = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [23]:
score

{'fit_time': array([6.04311013, 5.75637937, 2.63990593, 2.62183309, 1.7774291 ]),
 'score_time': array([0.01442432, 0.01424742, 0.01395631, 0.01354837, 0.01200294]),
 'test_score': array([0.875     , 0.87211538, 0.89701636, 0.8719923 , 0.87391723]),
 'train_score': array([0.9494828 , 0.94443108, 0.94468494, 0.94324194, 0.95045695])}

In [24]:
print('train 스코어 : ', np.mean(score['train_score']), '\ntest 스코어 : ', np.mean(score['test_score']))

train 스코어 :  0.9464595437171814 
test 스코어 :  0.8780082549788999


일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 좋은 성능을 얻을 수 있다.<br>
순서대로 트리를 추가를 하기 때문에 훈련 속도가 느리다.

### 히스토그램 기반 그레이디언트 부스팅
- 그레이디언트 부스팅의 속도와 성능을 더욱 개선한 모델.
- 정형 데이터를 다루는 머신러닝 알고리즘 가장 인기가 높다.
- 입력 특성 256개의 구간을 나눈다.
- 노드 분할할 때 최적의 분할을 매우 빠르게 찾을수 있다.
- 256개의 구간 중 하나를 떼어 놓고 누락된 값을 위해 사용. --> 전처리 안해도됨.

In [25]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
score = cross_validate(hgb, train_input, train_target, return_train_score=True)

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [26]:
score

{'fit_time': array([1.02986646, 0.52155232, 0.36639595, 0.36819243, 0.35377789]),
 'score_time': array([0.0240953 , 0.01086497, 0.0107801 , 0.02074742, 0.01091075]),
 'test_score': array([0.87115385, 0.88365385, 0.90279115, 0.86621752, 0.87680462]),
 'train_score': array([0.93192206, 0.93216262, 0.92857143, 0.93265993, 0.93554594])}

In [27]:
print('train 스코어 : ', np.mean(score['train_score']), '\ntest 스코어 : ', np.mean(score['test_score']))

train 스코어 :  0.9321723946453317 
test 스코어 :  0.8801241948619236


과적합을 줄익오 그레이디언트 부스팅보다 조금 더 좋은 성능.

In [31]:
# 특성 중요도를 계산 -> permutation_importance()
# 특성을 하나식 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰 후 중요한 특성인지 계산
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
# train 데이터에서 특성의 중요도 계산.
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

In [32]:
# importances(중요도), importances_mean(평균), importances_std(표준편차)
result.importances_mean

array([0.08876275, 0.23438522, 0.08027708])

In [33]:
# test 데이터에서 특성의 중요도 계산.
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)

In [34]:
result.importances_mean

array([0.05969231, 0.20238462, 0.049     ])

위 분석을 통해서 모델에서 어떤 특성에 관심을 둘지 예상 가능.

In [35]:
# 히스토그램 기반 그레이디언트 사용해서 테스트데이터 성능 확인
hgb.score(test_input, test_target)

0.8723076923076923

### XGBoost
- 교차검증 사용 가능.
- 그래디언트 부스팅 업그레이드 한 모델.

In [36]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
score = cross_validate(xgb, train_input, train_target, return_train_score=True)

In [37]:
print('train 스코어 : ', np.mean(score['train_score']), '\ntest 스코어 : ', np.mean(score['test_score']))

train 스코어 :  0.9558403027491312 
test 스코어 :  0.8782000074035686


### LightGBM
- 빠르고 최신 기술을 많이 적용

In [43]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42, force_col_wise=True)
score = cross_validate(lgb, train_input, train_target, return_train_score=True)

[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757816 -> initscore=1.140744
[LightGBM] [Info] Start training from score 1.140744
[LightGBM] [Info] Number 

In [44]:
print('train 스코어 : ', np.mean(score['train_score']), '\ntest 스코어 : ', np.mean(score['test_score']))

train 스코어 :  0.935828414851749 
test 스코어 :  0.8801251203079884
